### VeeCAD Parsing

Here are some initial notes and experiments regarding the parsing of VeeCAD's `*.per` files. The layout from the editor is stored as a `JSON` formatted file. It is possible to extract various information about the board layout, component footprints and positions, wires, nets, you name it. I'll include some basic information relevant to the current version `2.46.0.0`. Here's an accounting of the most useful stuff for this plugin imo:

```
'Board': {'Width': int, 'Height': int},
'*Outlines': [{}] # I'll discuss this later on,
'Components': [{ 'Designator': str, # Component ID
        'Outline': str, # outline name
        'X1000': int, # X position* of component * 1000
        'Y1000': int, # Y     "   * "      "     "  "
        'EndDeltaX': int, # rotation, kinda... more to come 
        'EndDeltaY': int, # ditto

}],
```

There's more categories at the top and throughout. These are not super useful to panelization, so I'll leave exploration for another time.

In [22]:
import json
import csv

In [ ]:
file_path = "../rotation_test.per"

try:
    # load json
    with open(file_path, 'r') as f:
        cad_text = f.readlines()
        json_data = ' '.join(cad_text[3:])

        json_data = json.loads(json_data)

        print("Data loaded succesfully!")

except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")
except json.JSONDecodeError:
    print(f"Error: Malformed JSON from file '{file_path}'")

json_data.keys()

Data loaded succesfully!


dict_keys(['Config', 'Board', 'CelledOutlines', 'LeadedOutlines', 'RadialOutlines', 'CustomOutlines', 'SmdOutlines', 'Components', 'Links', 'Breaks', 'Wires', 'Text', 'Nets', 'NetColors', 'Notes'])

#### Useful layers unwrapped

In [ ]:
json_data['Components']

[{'Designator': 'J1',
  'Value': '0',
  'Outline': 'THONK',
  'X1000': 5000,
  'Y1000': 3000,
  'EndDeltaX': 0,
  'EndDeltaY': 1,
  'Text': {'X': 1, 'Y': 1, 'Angle': 0, 'Visible': True},
  'Group': 0},
 {'Designator': 'J2',
  'Value': '90',
  'Outline': 'THONK',
  'X1000': 5000,
  'Y1000': 12000,
  'EndDeltaX': 1,
  'EndDeltaY': 0,
  'Text': {'X': 1, 'Y': 1, 'Angle': 0, 'Visible': True},
  'Group': 0},
 {'Designator': 'J3',
  'Value': '180',
  'Outline': 'THONK',
  'X1000': 7000,
  'Y1000': 19000,
  'EndDeltaX': 0,
  'EndDeltaY': -1,
  'Text': {'X': 1, 'Y': 1, 'Angle': 0, 'Visible': True},
  'Group': 0},
 {'Designator': 'J4',
  'Value': '270',
  'Outline': 'THONK',
  'X1000': 8000,
  'Y1000': 22000,
  'EndDeltaX': -1,
  'EndDeltaY': 0,
  'Text': {'X': 1, 'Y': 1, 'Angle': 0, 'Visible': True},
  'Group': 0},
 {'Designator': 'VR1',
  'Value': '0',
  'Outline': 'POT5_5',
  'X1000': 13000,
  'Y1000': 1000,
  'EndDeltaX': 0,
  'EndDeltaY': 1,
  'Text': {'X': 2, 'Y': 0, 'Angle': 0, 'Visible':

In [26]:
json_data['Components'][4]

{'Designator': 'VR1',
 'Value': '0',
 'Outline': 'POT5_5',
 'X1000': 13000,
 'Y1000': 1000,
 'EndDeltaX': 0,
 'EndDeltaY': 1,
 'Text': {'X': 2, 'Y': 0, 'Angle': 0, 'Visible': True},
 'Group': 0}

In [25]:
json_data['CelledOutlines']

[{'Name': 'POT5_5',
  'Locked': True,
  'Rows': [['Free', 'Body', 'Body', 'Body', 'Free'],
   ['Body', 'Body', 'Body', 'Body', 'Body'],
   ['Free', 'Body', 'Body', 'Body', 'Free'],
   ['Free', 'Body', 'Body', 'Body', 'Free'],
   ['Free', {'Pin': '1'}, {'Pin': '2'}, {'Pin': '3'}, 'Free']]},
 {'Name': 'THONK',
  'Locked': True,
  'Rows': [['Body', {'Pin': '1'}, 'Body'],
   ['Body', 'Body', 'Body'],
   ['Body', 'Body', 'Body'],
   ['Body', {'Pin': '2'}, 'Body'],
   ['Free', {'Pin': '3'}, 'Free']]}]

#### Data Extractions
Parsing out useful data regarding components, positions, and rotations.

First, we extract the names and first pin locations for all the celled outlines in the drawing. We will place the hole in the panel relative to the first pin's position

In [37]:
ks = ('CelledOutlines', 1, 'Rows')
first_pin_value = "{'Pin': '1'}"

# There's definitely a more elegant way to do this
# Extract coordinates of pin # 1
def extract_celled_pin(rows, pin: int = 1):
    for i, r in enumerate(rows):
        for j, c in enumerate(r):
            if not isinstance(c, str) and c['Pin'] == str(pin): return (i,j) 

def extract_celled_outline_dict(outlines):
    return {d['Name']: extract_celled_pin(d['Rows'], 1) for d in outlines}



print(f"{json_data[ks[0]][ks[1]]['Name']} first pin: {extract_celled_pin(json_data[ks[0]][ks[1]][ks[2]])}")

extract_celled_outline_dict(json_data['CelledOutlines'])


THONK first pin: (0, 1)


{'POT5_5': (4, 1), 'THONK': (0, 1)}

##### Positions and locations
Positions and locations are a little weird in VeeCAD, but I think for these celled components they're pretty easy to decypher.

`X1000` and `Y1000` represent the `x` and `y` positions of the component multiplied by 1000. For celled components, the position is measured against the top-left-most cell.

`EndDeltaX` and `EndDeltaY` represent a few things. For components with leads (like resistors, capacitors, etc.), these are used to represent both rotation, and ending lead positions. Hence, the `End` bit there. However, for our case we care only about rotation. Through brute force experimentation with this test example, I've found the following to be true:
```
(0, 1):  No rotation,
(1, 0):  90 degrees,
(0, -1): 180 degrees,
(-1, 0): 270 degrees,
```
for `(EndDeltaX, EndDelta,Y)`

In [44]:
# Some math functions

rotations = { # rotation matrices to line up panels
    (0,1): [[1,0],[0,1]],
    (1,0): [[0, -1], [1, 0]],
    (0, -1): [[-1, 0], [0, -1]],
    (-1, 0): [[0, 1], [-1, 0]],
}

def matmul(A: list[list[float]], x: list[float]) -> list[float]:
    return [sum([i*j for i,j in zip(row, x)]) for row in A]

matmul(rotations[(-1,0)], [4, 2])

[2, -4]

In [ ]:
# Lmao gotta love those but I realize now vectors in FreeCAD can be rotated on their own, so let's extract the angles


[2, -4]